In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn import preprocessing
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

In [2]:
dataset =  pd.DataFrame(arff.loadarff('data/4year.arff')[0])
dataset = dataset.drop(columns=["Attr37"])
dataset = dataset.dropna()
dataset.describe()

,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr55,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64
count,8091.000000,8091.000000,8091.000000,8091.000000,8.091000e+03,8091.000000,8091.000000,8091.000000,8091.000000,8091.000000,...,8.091000e+03,8091.000000,8091.000000,8091.000000,8091.000000,8091.000000,8091.000000,8.091000e+03,8091.000000,8091.000000
mean,0.048761,0.516031,0.180543,2.745438,8.105560e+01,0.065842,0.060676,2.739699,1.598251,0.464641,...,8.532902e+03,0.027347,-0.012690,0.980542,1.014384,92.263964,11.682148,3.095150e+02,7.014592,26.655851
std,0.183423,0.398833,0.377479,6.615317,1.231755e+04,5.053292,0.189032,8.635029,1.181109,0.400167,...,8.125526e+04,1.449081,9.676144,2.182341,28.758891,1996.909462,69.396327,1.469750e+04,9.441743,375.161393
min,-5.965500,0.002567,-12.135000,0.021061,-2.289900e+05,-117.420000,-5.965500,-1.848200,0.000469,-12.473000,...,-7.132200e+05,-121.000000,-597.420000,-0.086331,-284.380000,0.038646,-12.656000,1.082100e+00,0.000000,0.000000
25%,0.003552,0.275615,0.022997,1.051900,-5.036850e+01,0.000000,0.005163,0.448995,1.012300,0.301865,...,3.817250e+01,0.005436,0.012816,0.896680,0.000000,5.414750,4.437950,4.552900e+01,3.022000,2.035850
50%,0.042728,0.471530,0.192760,1.566000,-2.379000e+00,0.000000,0.050727,1.075600,1.145100,0.505810,...,1.271200e+03,0.042501,0.100640,0.960500,0.012904,9.501000,6.365400,7.556400e+01,4.827200,3.945300
75%,0.109865,0.678360,0.384555,2.688750,4.843100e+01,0.093481,0.126375,2.538450,1.889100,0.698495,...,5.677950e+03,0.106750,0.235660,0.995070,0.245140,19.305500,9.829600,1.207500e+02,8.008050,8.699000
max,3.123300,13.063000,0.991120,385.570000,1.034100e+06,322.200000,3.123300,388.590000,16.603000,1.577000,...,6.123700e+06,0.999260,226.760000,187.000000,1661.000000,161540.000000,4718.600000,1.301600e+06,337.290000,19721.000000


In [3]:
# get labels and data from the data set
labels = np.array(dataset['class'].values, dtype=np.int32)
data = dataset.loc[:, "Attr1":"Attr64"].values

# normalize the data
min_max_scaler = preprocessing.MinMaxScaler()
data = min_max_scaler.fit_transform(data)
    
# create a random order
order = np.arange(labels.size)
np.random.shuffle(order)
labels = labels[order]
data = data[order]

# split into test and train dataset
split_point = int(len(labels) * 0.8)
train_data = data[:split_point]
test_data = data[split_point:]
train_labels = labels[:split_point]
test_labels = labels[split_point:]

# create also a validation dataset
split_point = int(len(train_labels) * 0.9)
val_data = train_data[split_point:]
val_labels = train_labels[split_point:]
train_data = train_data[:split_point]
train_labels = train_labels[:split_point]

# calculate the class weights
num_of_bankrupt = np.sum(train_labels)
num_of_samples = len(train_labels)
class_w = {0: num_of_bankrupt/num_of_samples, 1: (num_of_samples-num_of_bankrupt)/num_of_samples}

In [4]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, input_dim=63, activation='tanh'))
#model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(16, input_dim=63, activation='tanh'))
#model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, 
          train_labels, 
          epochs=10, 
          batch_size=64, 
          validation_data=(val_data, val_labels), 
          class_weight=class_w)

Instructions for updating:
Colocations handled automatically by placer.
Train on 5824 samples, validate on 648 samples
Epoch 1/10
5824/5824 [==============================] - 1s 139us/sample - loss: 0.6961 - acc: 0.7835 - val_loss: 0.4185 - val_acc: 0.9769
Epoch 2/10
5824/5824 [==============================] - 0s 39us/sample - loss: 0.6847 - acc: 0.7704 - val_loss: 0.5326 - val_acc: 0.9769
Epoch 3/10
5824/5824 [==============================] - 0s 40us/sample - loss: 0.6721 - acc: 0.8954 - val_loss: 0.4144 - val_acc: 0.9769
Epoch 4/10
5824/5824 [==============================] - 0s 40us/sample - loss: 0.6720 - acc: 0.8154 - val_loss: 0.6452 - val_acc: 0.8009
Epoch 5/10
5824/5824 [==============================] - 0s 40us/sample - loss: 0.6706 - acc: 0.8535 - val_loss: 0.5632 - val_acc: 0.9414
Epoch 6/10
5824/5824 [==============================] - 0s 39us/sample - loss: 0.6645 - acc: 0.8250 - val_loss: 0.4200 - val_acc: 0.9691
Epoch 7/10
5824/5824 [==============================] - 0s

In [5]:
model.evaluate(test_data, test_labels, batch_size=64)

1619/1619 [==============================] - 0s 19us/sample - loss: 0.3275 - acc: 0.9568


[0.32748675977764047, 0.95676345]

In [6]:
bank_test_data = test_data[test_labels == 1]
bank_test_label = np.ones(len(bank_test_data))
model.evaluate(bank_test_data, bank_test_label, batch_size=64)

42/42 [==============================] - 0s 35us/sample - loss: 1.1498 - acc: 0.0952


[1.149758219718933, 0.0952381]

In [7]:
no_bank_test_data = test_data[test_labels == 0]
no_bank_test_label = np.zeros(len(no_bank_test_data))
model.evaluate(no_bank_test_data, no_bank_test_label, batch_size=64)

1577/1577 [==============================] - 0s 21us/sample - loss: 0.3056 - acc: 0.9797


[0.3055873341059942, 0.9797083]